In [1]:
import os
import sys
import torch

import numpy as np
import matplotlib.pyplot as plt

from torchmetrics import JaccardIndex
sys.path.insert(0, '..')
from loaders.datasets import AmsterdamDataset

In [2]:
def visualize(**images):
    """
    Plot images in one row
    """
    n_images = len(images)
    plt.figure(figsize=(20,8))
    for idx, (name, image) in enumerate(images.items()):
#         print(idx)
        plt.subplot(1, n_images, idx + 1)
        plt.xticks([]); 
        plt.yticks([])
        # get title from the parameter names
        plt.title(name.replace('_',' ').title(), fontsize=20)
        plt.imshow(image)
    plt.show()

In [3]:
model = torch.load(os.path.join('..', 'models', 'cnns', 'new unet', 'best_model.pth'))

# images = os.path.join('..', 'data', 'images')
# annotations = os.path.join('..', 'data', 'spectrum-batch-1', 'annotations.json')

images = os.path.join('..', 'data', 'self-annotated-sample', 'images')
annotations = os.path.join('..', 'data', 'self-annotated-sample', 'annotations.json')

data = AmsterdamDataset(images, annotations)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
%matplotlib inline

miou = 0
IoU = JaccardIndex(num_classes=2, reduction='none')

for i, (image, mask) in enumerate(data):
    mask = np.squeeze(mask)
    x = torch.from_numpy(np.transpose(image, (2, 0, 1)).astype('float32')).unsqueeze(0).to('cuda:0')
    
    pred = model(x).detach().squeeze().cpu()#.numpy()
#     pred = np.transpose(pred, (1, 2, 0))
    
#     visualize(original_image=image,
#               ground_truth=mask,
#               prediction=pred)
#     print(pred.dtype)
    miou += IoU(torch.as_tensor(pred).unsqueeze(0), torch.as_tensor(mask).unsqueeze(0))
    
print(miou / (i + 1))

tensor([0.9264, 0.0741])
